In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # es para enseñar todas las columnas del df
#pd.set_option('display.max_rows', None)     # es para enseñar todas las filas del df


import warnings
warnings.simplefilter('ignore')   # es para quitar warnings

## Cargo datos

In [2]:
# Cargo de datos del excel

'''stop = pd.read_excel('../data/selectsvistas/VIEW_TOPOLOGY_STOPS.xlsx')
fleet = pd.read_excel('../data/selectsvistas/view_FLEETS.xlsx')
lines_version = pd.read_excel('../data/selectsvistas/VIEW_TOPOLOGY_LINES_VERSION.xlsx')
stops_routes = pd.read_excel('../data/selectsvistas/VIEW_TOPOLOGY_STOPS_ROUTES.xlsx')
stops_version = pd.read_excel('../data/selectsvistas/VIEW_TOPOLOGY_STOPS_VERSION.xlsx')
lines = pd.read_excel('../data/selectsvistas/VIEW_TOPOOGY_LINES.xlsx')
routes = pd.read_excel('../data/selectsvistas/VIEW_TOPOOGY_ROUTES.xlsx')
vehicles = pd.read_excel('../data/selectsvistas/VIEW_VEHICLES.xlsx')'''

# Exporto a csv para leer mas rápido

'''# Exporto a csv

stop.to_csv('../data/topologia_lineas/stop.csv', sep=',', index=False)
fleet.to_csv('../data/topologia_lineas/fleet.csv', sep=',', index=False)
lines_version.to_csv('../data/topologia_lineas/lines_version.csv', sep=',', index=False)
stops_routes.to_csv('../data/topologia_lineas/stops_routes.csv', sep=',', index=False)
stops_version.to_csv('../data/topologia_lineas/stops_version.csv', sep=',', index=False)
lines.to_csv('../data/topologia_lineas/lines.csv', sep=',', index=False)
routes.to_csv('../data/topologia_lineas/routes.csv', sep=',', index=False)
vehicles.to_csv('../data/topologia_lineas/vehicles.csv', sep=',', index=False)
'''


"# Exporto a csv\n\nstop.to_csv('../data/topologia_lineas/stop.csv', sep=',', index=False)\nfleet.to_csv('../data/topologia_lineas/fleet.csv', sep=',', index=False)\nlines_version.to_csv('../data/topologia_lineas/lines_version.csv', sep=',', index=False)\nstops_routes.to_csv('../data/topologia_lineas/stops_routes.csv', sep=',', index=False)\nstops_version.to_csv('../data/topologia_lineas/stops_version.csv', sep=',', index=False)\nlines.to_csv('../data/topologia_lineas/lines.csv', sep=',', index=False)\nroutes.to_csv('../data/topologia_lineas/routes.csv', sep=',', index=False)\nvehicles.to_csv('../data/topologia_lineas/vehicles.csv', sep=',', index=False)\n"

In [3]:
# Leo todos los csv

stops = pd.read_csv('../data/topologia_lineas/stop.csv')
fleet = pd.read_csv('../data/topologia_lineas/fleet.csv')
lines_version = pd.read_csv('../data/topologia_lineas/lines_version.csv')
stops_routes = pd.read_csv('../data/topologia_lineas/stops_routes.csv')
stops_version = pd.read_csv('../data/topologia_lineas/stops_version.csv')
lines = pd.read_csv('../data/topologia_lineas/lines.csv')
routes = pd.read_csv('../data/topologia_lineas/routes.csv')
vehicles = pd.read_csv('../data/topologia_lineas/vehicles.csv')

## STOPS

Tabla de paradas donde están latitudes y longitudes, la **Primary Key** es **`StopId`**

In [4]:
# Elimino todos los duplicados entre 'LineId','RouteId','StopId', para quedarme con la última version de la parada.

stops = stops.drop_duplicates(subset='StopId', keep='last')

In [5]:
# Me cargo la columna Stopversion 

stops = stops.drop('StopsVersion', axis=1)

In [6]:
# Relleno los valores nulos en Description

stops.Description = stops.Description.fillna('NULL')

In [7]:
# Modifico 'NULL' por 'NoName' por da problemas al cargar csv

stops.Description[stops.Description == 'NULL'] = 'NoName'

In [8]:
stops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 125 to 297888
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StopId       3797 non-null   int64  
 1   Name         3797 non-null   object 
 2   ShortName    3797 non-null   int64  
 3   Description  3797 non-null   object 
 4   Latitude     3797 non-null   float64
 5   Longitude    3797 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 207.6+ KB


## ROUTES

Tabla de rutas donde **`RutasID`** es **Primary Key** y **`LineId`** es la **Foreing Key**

In [9]:
# Elimino todos los duplicados entre 'RouteId','LineId','PublicCode', para quedarme con la última version de la ruta.

routes = routes.drop_duplicates(subset=['RouteId','LineId'], keep='last')

In [10]:
# Creo un nuevo Primary Key como 'RutasID'

routes.RoutesVersion = np.arange(routes.shape[0])

routes.rename(columns = {'RoutesVersion':'RutasId'}, inplace = True)

In [11]:
routes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 518 entries, 6 to 14974
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RutasId     518 non-null    int32 
 1   RouteId     518 non-null    int64 
 2   LineId      518 non-null    int64 
 3   PublicCode  518 non-null    int64 
 4   Name        518 non-null    object
 5   Direction   518 non-null    int64 
dtypes: int32(1), int64(4), object(1)
memory usage: 26.3+ KB


## STOPS_ROUTES

Tabla intermedia entre stops y routes, con **foreing keys** **`RouteId`** y **`StopId`**

Revisar si elimino `LineId`

In [12]:
# Elimino todos los duplicados entre 'LineId','RouteId','StopId', para quedarme con la última version de la ruta.

stops_routes = stops_routes.drop_duplicates(subset=['LineId','RouteId','StopId'], keep='last')

In [13]:
# Me cargo la columna StopsRoutesVersion y LineId

stops_routes = stops_routes.drop('StopsRoutesVersion', axis=1)

In [14]:
# Hago un left join entre 'stops_routes'  y 'routes' para obtener 'RutasId'


stops_routes = stops_routes.set_index(['LineId', 'RouteId']).join(routes[['LineId', 'RouteId', 'RutasId' ]].set_index(['LineId', 'RouteId']))


In [15]:
# Le elimino duplicado que se generan

stops_routes.drop_duplicates(inplace=True)

In [16]:
# Le reseteo el indice

stops_routes.reset_index(inplace=True)

In [17]:
stops_routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15316 entries, 0 to 15315
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LineId               15316 non-null  int64  
 1   RouteId              15316 non-null  int64  
 2   StopId               15316 non-null  int64  
 3   OrderInRoute         15316 non-null  int64  
 4   DistanceToFirstStop  15316 non-null  float64
 5   RutasId              15316 non-null  int32  
dtypes: float64(1), int32(1), int64(4)
memory usage: 658.2 KB


## LINES

Tabla de lineas donde **`LineId`** es la **Primary key** y donde **`FleetId`** es la **Foreing key**

In [18]:
# Elimino todos los duplicados entre 'RouteId','LineId','PublicCode', para quedarme con la última version de la ruta.

lines = lines.drop_duplicates(subset=['LineId'], keep='last')

In [19]:
# Me cargo la columna LinesVersion

lines = lines.drop('LinesVersion', axis=1)

In [20]:
lines.head()

,LineId,FleetId,PublicCode,Name
6,1,1,L1,6631|L1 MOUQUAOUAMA SUD INBIAT - HOPITAL HASSA...
59,2,1,L2,6632|L2 WILAYA - HAY MOHAMMEDI
112,3,1,L3,6633|L3 WILAYA VALLEE - ELHOUDA
119,4,1,L4,6634|L4 MOUQUAOUAMA SUD INBIAT - EL HASSANIA ANZA
172,5,1,L5,6635|L5 WILAYA VALLEE - DRARGA


## FLEETS

Tabla de la delegación de la flota **`FleetId`** es la **Primary key**

In [21]:
fleet.head()

,FleetId,FleetName,FleetNumber
0,1,669|Agadir,65
1,2,662|Rabat,66


## VEHICLES

Tabla de detalle de vehículos **`FleetId`** es la **Primary key**

In [22]:
vehicles.head()

,VehicleId,VehicleCode,FleetId,Plate,Model
0,1,9501,1,59697-A-33,K230 UB 4X2
1,2,9502,1,59675-A-33,K230 UB 4X2
2,3,9503,1,59691-A-33,K230 UB 4X2
3,4,9504,1,59868-A-33,K94UB 4X2
4,5,9505,1,59123-A-33,K230 UB 4X2


## Exporto datos

In [23]:
# Exporto a csv los archivos modificados para cargar en base de datos.

stops.to_csv('../data/red_transporte/stops.csv', sep=',', index=False)
fleet.to_csv('../data/red_transporte/fleet.csv', sep=',', index=False)
stops_routes.to_csv('../data/red_transporte/stops_routes.csv', sep=',', index=False)
lines.to_csv('../data/red_transporte/lines.csv', sep=',', index=False)
routes.to_csv('../data/red_transporte/routes.csv', sep=',', index=False)
vehicles.to_csv('../data/red_transporte/vehicles.csv', sep=',', index=False)
